In [1]:
from torchmetrics.functional import auc, mean_squared_error
from torchmetrics import F1Score
from tools import *
from CONSTANT import *
from models import CNNBiLSTM, CNNTransformer
from config import Params
from torch.utils.data import (
    TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import time
import datetime
import os
import sys
import time
import warnings

# load baseline results

In [2]:
bs_results = load_dict_model(r'./output/KEC/valence_CTransformer_loso_0.0001_64_32/results.pkl')
parse_res(bs_results)
# valence CTransformer

0.56804

In [3]:
bs_results = load_dict_model(r'./output/KEC/arousal_CLSTM_loso_0.0001_64_32/results.pkl')
parse_res(bs_results)
# arousal CLSTM

0.9817

# set params

In [4]:
ckpt_path = r'./output/HKU956/valence_CTransformer_loso_0.0001_256_32/fold1_checkpoint.pt'

args = Params(dataset='KEC', 
              model='CTransformer',
              target='arousal', 
              debug=False, 
              fcn_input=12608,
              batch_size=64
              )

# load data

In [5]:
spliter = load_model(args.spliter)
data = pd.read_pickle(args.data)

for i, k in enumerate(spliter[args.valid]):
    train_index = k['train_index']
    test_index = k['test_index']
    break

dataprepare = DataPrepare(args,
                        target=args.target, data=data, train_index=train_index, test_index=test_index, device=args.device, batch_size=args.batch_size
                        )

train_dataloader, test_dataloader = dataprepare.get_data()

(2837, 4, 400) (2837, 1) (608, 4, 400) (608, 1)


# load pretrain model

In [6]:
kec_fcn = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(args.fcn_input, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

In [7]:
model = CNNTransformer.CTransformer(args)
model.load_state_dict(torch.load(ckpt_path))
model.fcn = kec_fcn
model = model.to(args.device)

# train and eval

In [8]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss_list = []
    loss_fn = nn.MSELoss()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target.float())
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
    return np.mean(train_loss_list)


def eval(model, device, val_loader):
    model.eval()
    val_loss = []
    loss_fn = nn.MSELoss()
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target.float())
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [9]:
def run(train_loader, val_loader, ckpt_path):
    best_score = float('inf')
    patience = 25
    stop_count = 0
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                                    verbose=True, threshold_mode='rel',
                                    cooldown=0, min_lr=0, eps=1e-08
                                    )
    for epoch in range(1, args.epochs + 1):
        train_loss = train(model, args.device, train_loader, optimizer, epoch)
        val_loss = eval(model, args.device, val_loader)
        scheduler.step(val_loss)
        print('[Epoch{}] | train_loss:{:.4f} | val_loss:{:.4f} | lr:{:e}'.format(epoch, train_loss, val_loss, optimizer.param_groups[0]['lr']))

        if val_loss < best_score:
            best_score = val_loss
            torch.save(model.state_dict(), ckpt_path)
            print("<<<<<< reach best {0} >>>>>>".format(val_loss))
            stop_count = 0
        else:
            model.load_state_dict(torch.load(ckpt_path))
            stop_count += 1
            if stop_count >= patience:
                print("<<<<<< without improvement in {} epoch, early stopping, best score {:.4f} >>>>>>".format(patience, best_score))
                break
        # wandb.log({'train_loss': train_loss, 'val_loss': val_loss})
    print('best score', best_score)
    return model

In [10]:
model = run(train_dataloader, test_dataloader, ckpt_path=os.path.join(args.save_path, 'arousal_checkpoint_dp02.pt'))

45it [00:06,  7.31it/s]


[Epoch1] | train_loss:1.6805 | val_loss:1.3359 | lr:1.000000e-04
<<<<<< reach best 1.3358946472406388 >>>>>>


45it [00:03, 12.38it/s]


[Epoch2] | train_loss:1.2357 | val_loss:1.7690 | lr:1.000000e-04


45it [00:03, 12.43it/s]


...

[Epoch146] | train_loss:0.9721 | val_loss:1.1946 | lr:1.220703e-08


45it [00:03, 12.38it/s]


[Epoch147] | train_loss:0.9690 | val_loss:1.1946 | lr:1.220703e-08


45it [00:03, 12.38it/s]


[Epoch148] | train_loss:0.9532 | val_loss:1.1946 | lr:1.220703e-08
<<<<<< without improvement in 25 epoch, early stopping, best score 1.1946 >>>>>>
best score 1.1946289479732513
